First, import the fish and modules.

In [1]:
import inspectable_fish
from mayavi import mlab
fish_labels = ['2015-07-11-1 Chena - Chinook Salmon (id #4)']
fishes = [inspectable_fish.InspectableFish(fish_label) for fish_label in fish_labels]
for fish in fishes:
    fish.load_state()

/Users/Jason/anaconda/envs/driftmodelenv/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


Then, assign movie_fish to the subject of the movie and plot its predicted detection field. Manually remove the toolbar at the top (right-clicking) and pan/zoom the scene into the best view for rotating around in the video. Then, hit "stop" on the popup window to stop interacting with it, which will stop this cell's execution and make executing the next one possible.

In [3]:
movie_fish = fishes[0]
fig3d = movie_fish.plot_predicted_detection_field(gridsize=70j, surfaces=False, velocities=False, stopButton=True)

/Users/Jason/anaconda/envs/driftmodelenv/lib/python3.5/site-packages/numpy/lib/function_base.py:2815: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


/Users/Jason/anaconda/envs/driftmodelenv/lib/python3.5/site-packages/scipy/interpolate/interpolate.py:595: RuntimeWarning: divide by zero encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
/Users/Jason/anaconda/envs/driftmodelenv/lib/python3.5/site-packages/scipy/interpolate/interpolate.py:598: RuntimeWarning: invalid value encountered in multiply
  y_new = slope*(x_new - x_lo)[:, None] + y_lo


In [12]:
movie_fish = fishes[0]
movie_fish.cforager.print_parameters()

ff = movie_fish.cforager.get_parameter_named("flicker_frequency")
tau_0 = movie_fish.cforager.get_parameter_named("tau_0")
delta_0 = movie_fish.cforager.get_parameter_named("delta_0")

movie_fish.cforager.set_parameter(ff, 70)
movie_fish.cforager.set_parameter(tau_0, 1e-1)
movie_fish.cforager.set_parameter(delta_0, 1)
fig3d = movie_fish.plot_predicted_detection_field(gridsize=150j, show_fielddata=False, otf_exponent=0.3, surfaces=False, velocities=False, stopButton=True)

/Users/Jason/anaconda/envs/driftmodelenv/lib/python3.5/site-packages/numpy/lib/function_base.py:2815: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


/Users/Jason/anaconda/envs/driftmodelenv/lib/python3.5/site-packages/scipy/interpolate/interpolate.py:595: RuntimeWarning: divide by zero encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
/Users/Jason/anaconda/envs/driftmodelenv/lib/python3.5/site-packages/scipy/interpolate/interpolate.py:598: RuntimeWarning: invalid value encountered in multiply
  y_new = slope*(x_new - x_lo)[:, None] + y_lo


Finally, execute this one -- after hitting 'stop' in the previous one -- to export the movie.

Run the result through Wondershare Pro (not Compressor) to convert it into a format that Quicktime and other players don't choke on. It can also play immediately in VLC.

In [13]:
import moviepy.video.VideoClip as vc
import os
VIDEO_EXPORT_PATH = "/Users/Jason/Desktop/"
fps = 60
revolution_time = 15
revolutions = 1
def make_frame(t):
    current_view = mlab.view(figure=fig3d)
    mlab.view(azimuth=90 + (t / revolution_time) * 360, elevation=current_view[1], distance=current_view[2], focalpoint=current_view[3], figure=fig3d)
    return mlab.screenshot(antialiased=True, figure=fig3d)
frame = make_frame(1)
rotating_movie = vc.VideoClip(make_frame, duration=revolution_time * revolutions - 1 / fps)
rotating_movie.write_videofile(VIDEO_EXPORT_PATH + movie_fish.label + ".mp4", fps=fps, codec='mpeg4', bitrate='8000k', audio=False)
mlab.close()
os.system('open "%s"' % VIDEO_EXPORT_PATH)

/Users/Jason/anaconda/envs/driftmodelenv/lib/python3.5/site-packages/mayavi-4.5.0-py3.5-macosx-10.6-x86_64.egg/mayavi/tools/camera.py:288: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if focalpoint is not None and not focalpoint == 'auto':


[MoviePy] >>>> Building video /Users/Jason/Desktop/2015-07-11-1 Chena - Chinook Salmon (id #4).mp4


[MoviePy] Writing video /Users/Jason/Desktop/2015-07-11-1 Chena - Chinook Salmon (id #4).mp4


  0%|          | 0/900 [00:00<?, ?it/s]

/Users/Jason/anaconda/envs/driftmodelenv/lib/python3.5/site-packages/mayavi-4.5.0-py3.5-macosx-10.6-x86_64.egg/mayavi/tools/camera.py:288: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if focalpoint is not None and not focalpoint == 'auto':


...

100%|█████████▉| 898/900 [10:34<00:01,  1.11it/s]

100%|█████████▉| 899/900 [10:35<00:00,  1.11it/s]

[MoviePy] Done.


[MoviePy] >>>> Video ready: /Users/Jason/Desktop/2015-07-11-1 Chena - Chinook Salmon (id #4).mp4 



0